In [38]:
!pip3 install torch torchvision torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 4.9 MB/s eta 0:00:00


In [68]:
import torch
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(14, 7),
    nn.ReLU(),
    nn.Linear(7, 7),
    nn.ReLU(),
    nn.Linear(7, 7),
    nn.ReLU(),
    nn.Linear(7, 1),
    nn.Sigmoid(),
    nn.Flatten(start_dim=0)
)



In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchmetrics import PrecisionRecallCurve
import torch.optim as optim


display_labels = {"No Diabetic", "Yes Diabetic"}



# Seperating Features (X) AND target Variable (Y)
data = pd.read_csv("cleaned_diabetes_one_hot_encoding.csv")

gk = data.groupby('diabetes')

diabetes_true = gk.get_group(1)
diabetes_false = gk.get_group(0)
diabetes_false = diabetes_false.sample(8028)
print(diabetes_true.shape[0])
print(diabetes_false.shape[0])
data = pd.concat([diabetes_false, diabetes_true])
print(data.shape[0])
X = data.drop(['diabetes'], axis=1)
Y = data['diabetes']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

X_train = torch.from_numpy(X_train.values).float().to("cpu")
y_train = torch.from_numpy(y_train.values).float().to("cpu")
X_val = torch.from_numpy(X_val.values).float().to("cpu")
y_val = torch.from_numpy(y_val.values).float().to("cpu")
X_test = torch.from_numpy(X_test.values).float().to("cpu")
y_test = torch.from_numpy(y_test.values).float().to("cpu")

batch_size=16
epochs = round(len(X_train)/64)

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# collect statistics
train_loss = []
train_acc = []
val_acc = []
TP = 0
FP = 0
FN = 0
diabetes_bias=0.1
for epoch in range(0, epochs):
  for i in range(0, batch_size):
        start = i * batch_size
        # take a batch
        Xbatch = X_train[start:start+batch_size]
        ybatch = y_train[start:start+batch_size]
        # forward pass
        y_pred = model(Xbatch)
        #print(y_pred)
        loss = loss_fn(y_pred, ybatch)
        acc = (y_pred.round() == ybatch).float().mean()
        #metric = BinaryPrecisionRecallCurve(y_pred, ybatch)
        y_pred_round = y_pred.round().int()
        y_batch_int = ybatch.int()
        true_result = torch.bitwise_and(y_pred_round, y_batch_int)
        diff = torch.subtract(y_pred_round, y_batch_int)
        for elm in diff:
          if elm == -1:
            FN += 1
          elif elm == 1:
            FP += 1
          elif elm == 0:
            TP += 1


        train_loss.append(float(loss))
        train_acc.append(float(acc))

        optimizer.zero_grad()
        loss.backward()
        # Weight
        optimizer.step()

  y_pred = model(X_val)
  acc = (y_pred.round() == y_val).float().mean()
  val_acc.append(float(acc))
  print(f"End of {epoch}, accuracy {acc} Precision {TP/(TP+FP)} Recall {TP/(TP+FN)} F1 {TP/(TP+1/2*(FP+FN))}")
  #print(FN)
  #print(FP)
  #print(TP)
  torch.save(model, "diabettes_classifier.pt")




8028
8028
16056
End of 0, accuracy 0.8767123222351074 Precision 0.9469387755102041 Recall 0.9547325102880658 F1 0.9508196721311475
End of 1, accuracy 0.8922789692878723 Precision 0.9468302658486708 Recall 0.9526748971193416 F1 0.9497435897435897
End of 2, accuracy 0.8829389810562134 Precision 0.9508867667121419 Recall 0.9521857923497268 F1 0.9515358361774744
End of 3, accuracy 0.892901599407196 Precision 0.950920245398773 Recall 0.9528688524590164 F1 0.9518935516888434
End of 4, accuracy 0.8891656398773193 Precision 0.9517184942716858 Recall 0.9524979524979525 F1 0.9521080638559148
End of 5, accuracy 0.8891656398773193 Precision 0.9522835719154737 Recall 0.9529331514324693 F1 0.9526082509376066
End of 6, accuracy 0.8897882699966431 Precision 0.9526869158878505 Recall 0.9532437171244886 F1 0.9529652351738241
End of 7, accuracy 0.8885429501533508 Precision 0.9535002554931017 Recall 0.9535002554931017 F1 0.9535002554931017
End of 8, accuracy 0.8897882699966431 Precision 0.9536784741144414